In [1]:
# !pip install feature-engine

In [2]:
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
from colorama import Back, Fore, Style
from feature_engine.encoding import WoEEncoder
from scipy.stats import rankdata
from sklearn.impute import KNNImputer
from sklearn.linear_model import (HuberRegressor, LinearRegression,
                                  LogisticRegression)
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler

warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [4]:
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m4_missing'] = data['measurement_4'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']
    data['loading'] = np.log(data['loading'])

    full_fill_dict = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    features = [f for f in test.columns if 'measurement' in f] + ['loading']
    for code in data.product_code.unique():
        tmp = data[data.product_code==code]
        column = full_fill_dict[code]
        tmp_train = tmp[column+['measurement_17']].dropna(how='any')
        tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp['measurement_17'].isnull())]
        print(f"code {code} has {len(tmp_test)} samples to fill nan")
        model = HuberRegressor()
        model.fit(tmp_train[column], tmp_train['measurement_17'])
        data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data['measurement_17'].isnull()), 'measurement_17'] = model.predict(tmp_test[column])
        model2 = KNNImputer(n_neighbors=5)
        print(f"KNN imputing code {code}")
        data.loc[data.product_code==code, features] = model2.fit_transform(data.loc[data.product_code==code, features])
    
    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    
    df_train = data.iloc[:df_train.shape[0], :]
    df_test = data.iloc[df_train.shape[0]:, :]
    
    woe_encoder = WoEEncoder(variables=['attribute_0'])
    woe_encoder.fit(df_train, df_train['failure'])
    df_train = woe_encoder.transform(df_train)
    df_test = woe_encoder.transform(df_test)

    return df_train, df_test

df_train, df_test = preprocessing(train, test)

code A has 386 samples to fill nan
KNN imputing code A
code B has 418 samples to fill nan
KNN imputing code B
code C has 391 samples to fill nan
KNN imputing code C
code D has 398 samples to fill nan
KNN imputing code D
code E has 429 samples to fill nan
KNN imputing code E
code F has 420 samples to fill nan
KNN imputing code F
code G has 373 samples to fill nan
KNN imputing code G
code H has 361 samples to fill nan
KNN imputing code H
code I has 377 samples to fill nan
KNN imputing code I


In [5]:
def test_scale(test_data, feats):
    scaler = StandardScaler()

    scaled_test = scaler.fit_transform(test_data[feats])
    
    #back to dataframe
    new_test = test_data.copy()
    
    new_test[feats] = scaled_test
    
    assert len(test_data) == len(new_test)
    
    return new_test

In [6]:
select_features = {
    0: ['loading', 'measurement_17', 'm5_missing', 'attribute_0'],
    1: ['loading', 'measurement_17', 'm5_missing', 'area'],
    2: ['loading', 'measurement_17', 'm5_missing', 'm3_missing'],
    3: ['loading', 'measurement_17', 'm5_missing', 'measurement_2']
}

In [7]:
test.columns

Index(['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17'],
      dtype='object')

In [8]:
x_test = df_test.copy()
for clf_k in range(4):
    filename = f'Model{clf_k}.sav'
    current_model = pickle.load(open(filename, 'rb'))
    print(f'############## Classifier {clf_k} #################')
    features = select_features[clf_k]
    x_test = test_scale(x_test, features)
    lr_test = current_model.predict_proba(x_test[features])[:, -1]
    submission[f'lr{clf_k}'] = lr_test

############## Classifier 0 #################
############## Classifier 1 #################
############## Classifier 2 #################
############## Classifier 3 #################


In [9]:
submission['rank0'] = rankdata(submission['lr0'])
submission['rank1'] = rankdata(submission['lr1'])
submission['rank2'] = rankdata(submission['lr2'])
submission['rank3'] = rankdata(submission['lr3'])

In [10]:
submission['failure'] = submission['rank0']*0.2 + submission['rank1']*0.25 + submission['rank2']*0.25 + submission['rank3']*0.3

In [11]:
submission.head()

,id,failure,lr0,lr1,lr2,lr3,rank0,rank1,rank2,rank3
0,26570,8735.15,0.207813,0.210654,0.209319,0.209060,7915.0,9519.0,8748.0,8618.0
1,26571,5630.60,0.202801,0.205590,0.204278,0.200867,5354.0,6740.0,6092.0,4506.0
2,26572,7335.25,0.205529,0.208346,0.207023,0.205701,6744.0,8196.0,7525.0,6854.0
3,26573,7038.20,0.204016,0.206818,0.205501,0.207375,5952.0,7409.0,6735.0,7706.0
4,26574,20155.55,0.239534,0.242677,0.241205,0.242120,19982.0,20258.0,20135.0,20203.0


In [12]:
submission[['id', 'failure']].to_csv('submission.csv', index=False)